# Calibration Tutorial - Fort Peck, MT - Unirrigated Flux Plot

## Step 3: Running the Calibrated Model

Now we evaluate whether calibration improved model performance by running in **forecast mode** with calibrated parameters.

This notebook:
1. Visualizes how parameters evolved during calibration
2. Runs the model with calibrated parameters
3. Compares calibrated vs uncalibrated performance against flux observations

In [ ]:
import os
import sys
import time

import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score

root = os.path.abspath('../..')
sys.path.append(root)

from swimrs.swim.config import ProjectConfig
from swimrs.swim.sampleplots import SamplePlots
from swimrs.model.obs_field_cycle import field_day_loop

from swimrs.viz.param_evolution import plot_parameter_histograms
from swimrs.viz.swim_timeseries import plot_swim_timeseries

%matplotlib inline

In [ ]:
project_ws = os.path.abspath('.')
data = os.path.join(project_ws, 'data')
pest_dir = os.path.join(project_ws, 'pest')

config_file = os.path.join(project_ws, '2_Fort_Peck.toml')

## 1. Visualize Parameter Evolution

Let's see how the parameters changed across optimization iterations. The histograms show the distribution of parameter values across ensemble realizations.

In [ ]:
initial_params = os.path.join(project_ws, 'params.csv')

# Get all parameter files from optimization steps
steps = []
for i in range(10):  # Check up to 10 iterations
    step_file = os.path.join(pest_dir, f'2_Fort_Peck.{i}.par.csv')
    if os.path.exists(step_file):
        steps.append(step_file)

if steps:
    print(f"Found {len(steps)} optimization steps")
    
    fig_dir = os.path.join(project_ws, 'figures', 'parameter_hist')
    os.makedirs(fig_dir, exist_ok=True)
    
    # Plot histograms (set fig_out_dir=fig_dir to save PNGs)
    plot_parameter_histograms(initial_params, steps, fig_out_dir=None)
else:
    print("No parameter files found. Run notebook 02_calibration first.")

## 2. Run the Calibrated Model

To run with calibrated parameters:
1. Set `forecast=True` when reading config
2. Ensure `[forecast]` section in config points to the final `.par.csv` file

The config file should have:
```toml
[forecast]
forecast_parameters = "{pest_run_dir}/pest/2_Fort_Peck.3.par.csv"
```

In [ ]:
def run_fields(ini_path, project_ws, selected_feature, output_csv, forecast=False):
    """Run SWIM model and save combined input/output to CSV."""
    start_time = time.time()

    config = ProjectConfig()
    config.read_config(ini_path, project_ws, forecast=forecast)

    fields = SamplePlots()
    fields.initialize_plot_data(config)
    fields.output = field_day_loop(config, fields, debug_flag=True)

    end_time = time.time()
    print(f'\nExecution time: {end_time - start_time:.2f} seconds\n')

    out_df = fields.output[selected_feature].copy()
    in_df = fields.input_to_dataframe(selected_feature)
    df = pd.concat([out_df, in_df], axis=1, ignore_index=False)
    df.to_csv(output_csv)
    
    return df

In [ ]:
selected_feature = 'US-FPe'
out_csv = os.path.join(project_ws, f'combined_output_{selected_feature}_calibrated.csv')

# Run with forecast=True to use calibrated parameters
df = run_fields(config_file, project_ws, selected_feature=selected_feature, 
                output_csv=out_csv, forecast=True)

## 3. Visualize Calibrated Output

In [ ]:
ydf = df.loc['2004-01-01': '2004-12-31']
print(f'Total irrigation: {ydf.irrigation.sum():.1f} mm')
print(f'Total ET: {ydf.et_act.sum():.1f} mm')
print(f'Total precipitation: {ydf.ppt.sum():.1f} mm')

plot_swim_timeseries(ydf, ['et_act', 'etref', 'rain', 'melt', 'irrigation'], 
                     start='2004-01-01', end='2004-12-31', png_dir='et_calibrated.png')

## 4. Compare with Flux Tower Observations

In [ ]:
def compare_etf_estimates(combined_output_path, flux_data_path, irr=False):
    """Compare model Kc_act and SSEBop ETf against flux tower observations."""
    flux_data = pd.read_csv(flux_data_path)
    flux_data = flux_data.set_index(pd.to_datetime(flux_data['date']))['EToF']

    combined_output = pd.read_csv(combined_output_path, index_col=0)
    combined_output.index = pd.to_datetime(combined_output.index)

    if irr:
        etf, ct = 'etf_irr', 'etf_irr_ct'
    else:
        etf, ct = 'etf_inv_irr', 'etf_inv_irr_ct'

    df = pd.DataFrame({'kc_act': combined_output['kc_act'],
                       'etf': combined_output[etf],
                       'ct': combined_output[ct],
                       'EToF': flux_data})

    # Filter for days that have a SSEBop ETf retrieval and a flux observation
    df = df.dropna()
    df = df.loc[df['ct'] == 1]

    # Calculate RMSE and R-squared
    rmse_kc_act = np.sqrt(mean_squared_error(df['EToF'], df['kc_act']))
    r2_kc_act = r2_score(df['EToF'], df['kc_act'])

    rmse_ssebop = np.sqrt(mean_squared_error(df['EToF'], df['etf']))
    r2_ssebop = r2_score(df['EToF'], df['etf'])

    print(f"SWIM Kc_act vs. Flux EToF: RMSE = {rmse_kc_act:.2f}, R-squared = {r2_kc_act:.2f}")
    print(f"SSEBop ETf vs. Flux EToF: RMSE = {rmse_ssebop:.2f}, R-squared = {r2_ssebop:.2f}")
    
    return df, rmse_kc_act, r2_kc_act

In [ ]:
flux_data = os.path.join(data, 'US-FPe_daily_data.csv')
comparison_df, rmse_cal, r2_cal = compare_etf_estimates(out_csv, flux_data)

### Compare with Uncalibrated Results

In [ ]:
uncal_csv = os.path.join(project_ws, f'combined_output_{selected_feature}_uncalibrated.csv')

if os.path.exists(uncal_csv):
    print("Uncalibrated model:")
    _, rmse_uncal, r2_uncal = compare_etf_estimates(uncal_csv, flux_data)
    
    print(f"\n" + "="*60)
    print("IMPROVEMENT SUMMARY")
    print("="*60)
    print(f"RMSE reduction: {rmse_uncal:.2f} -> {rmse_cal:.2f} ({(rmse_uncal-rmse_cal)/rmse_uncal*100:.1f}% improvement)")
    print(f"R-squared:      {r2_uncal:.2f} -> {r2_cal:.2f}")
else:
    print("Uncalibrated output not found. Run notebook 01 first.")

In [ ]:
import matplotlib.pyplot as plt

# Create scatter plot comparison
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

ax1 = axes[0]
ax1.scatter(comparison_df['EToF'], comparison_df['kc_act'], alpha=0.5, s=10)
ax1.plot([0, 1.5], [0, 1.5], 'r--', label='1:1 line')
ax1.set_xlabel('Flux EToF')
ax1.set_ylabel('SWIM Kc_act')
ax1.set_title(f'SWIM vs Flux (Calibrated)\nRMSE={rmse_cal:.2f}, R2={r2_cal:.2f}')
ax1.legend()
ax1.set_xlim(0, 1.5)
ax1.set_ylim(0, 1.5)

ax2 = axes[1]
ax2.scatter(comparison_df['EToF'], comparison_df['etf'], alpha=0.5, s=10)
ax2.plot([0, 1.5], [0, 1.5], 'r--', label='1:1 line')
ax2.set_xlabel('Flux EToF')
ax2.set_ylabel('SSEBop ETf')
ax2.set_title('SSEBop vs Flux')
ax2.legend()
ax2.set_xlim(0, 1.5)
ax2.set_ylim(0, 1.5)

plt.tight_layout()
plt.savefig('comparison_scatter_calibrated.png', dpi=150)
plt.show()

## Key Insights

Calibration significantly improved model performance:

- **RMSE reduced by >40%** from the uncalibrated model
- **SWIM now outperforms SSEBop** in daily ET estimation

### Why does this work?

**The key insight is that we can mine the deep remote sensing-based ET record, but rather than driving the model with remote sensing ET directly, we drive the calibration with it.**

The model has access to:
1. Daily meteorological data (not just satellite overpass days)
2. Physically-based soil water balance constraints
3. Flexibility to tune parameters using the remote sensing record

This combination gives SWIM a more grounded perspective on daily fluxes than remote sensing alone, resulting in better ET estimates.

### Practical implications

- **For data-sparse regions:** Calibrate once using available remote sensing, then apply calibrated parameters to generate daily ET
- **For irrigated lands:** Same approach works with mask-switching between irrigated/non-irrigated periods
- **For regional applications:** See Tutorial 4 (Flux Network) for scaling this approach